In [1]:
import re
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DoubleType,
)

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Teen_Addictions")
    .config("spark.executor.memory", "4g")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
# Tip to reader: use WARN for development, ERROR in prod

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 07:31:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv(
    path="dataset/teen_phone_addiction_dataset.csv",
    header=True,
    mode="PERMISSIVE",
    inferSchema=True
)

In [4]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- School_Grade: string (nullable = true)
 |-- Daily_Usage_Hours: double (nullable = true)
 |-- Sleep_Hours: double (nullable = true)
 |-- Academic_Performance: integer (nullable = true)
 |-- Social_Interactions: integer (nullable = true)
 |-- Exercise_Hours: double (nullable = true)
 |-- Anxiety_Level: integer (nullable = true)
 |-- Depression_Level: integer (nullable = true)
 |-- Self_Esteem: integer (nullable = true)
 |-- Parental_Control: integer (nullable = true)
 |-- Screen_Time_Before_Bed: double (nullable = true)
 |-- Phone_Checks_Per_Day: integer (nullable = true)
 |-- Apps_Used_Daily: integer (nullable = true)
 |-- Time_on_Social_Media: double (nullable = true)
 |-- Time_on_Gaming: double (nullable = true)
 |-- Time_on_Education: double (nullable = true)
 |-- Phone_Usage_Purpose: string 

## Data Cleaning and Transformations

### Changing Column Names to `snake_case`

In [5]:
def to_snake_case(column_name):
    """
    Convert column name to snake_case (PEP8 style).
    Example: "Addiction_level" -> "addiction_level"
    """
    
    name = column_name.lower()
    return name

In [6]:
current_columns = df.columns
column_mappings = {col: to_snake_case(col) for col in current_columns}

for old_name, new_name in column_mappings.items():
    df = df.withColumnRenamed(old_name, new_name)

In [7]:
df.show(3)

+---+---------------+---+------+------------+------------+-----------------+-----------+--------------------+-------------------+--------------+-------------+----------------+-----------+----------------+----------------------+--------------------+---------------+--------------------+--------------+-----------------+-------------------+--------------------+-------------------+---------------+
| id|           name|age|gender|    location|school_grade|daily_usage_hours|sleep_hours|academic_performance|social_interactions|exercise_hours|anxiety_level|depression_level|self_esteem|parental_control|screen_time_before_bed|phone_checks_per_day|apps_used_daily|time_on_social_media|time_on_gaming|time_on_education|phone_usage_purpose|family_communication|weekend_usage_hours|addiction_level|
+---+---------------+---+------+------------+------------+-----------------+-----------+--------------------+-------------------+--------------+-------------+----------------+-----------+----------------+----

### Working With Genders

In [8]:
df.select('gender').distinct().show()

+------+
|gender|
+------+
|Female|
| Other|
|  Male|
+------+



Did not know that we also have new gender, which is called `other`. New gender!

In [9]:
df = df.filter(F.col('gender') != 'Other')

## Exploration

In [10]:
df.createOrReplaceTempView('teen_dataset')

### I. Top 5 Addicted Teens

In [11]:
spark.sql(
    """
    select id, name, age, gender, school_grade, daily_usage_hours
    from 
        (select *, 
            dense_rank() over(partition by gender order by addiction_level desc) as rnk
        from teen_dataset) t
    where rnk <= 5
    """
).show()

+---+----------------+---+------+------------+-----------------+
| id|            name|age|gender|school_grade|daily_usage_hours|
+---+----------------+---+------+------------+-----------------+
|  1| Shannon Francis| 13|Female|         9th|              4.0|
|  2| Scott Rodriguez| 17|Female|         7th|              5.5|
| 11|    Bobby Sparks| 18|Female|        12th|              4.9|
| 13|    Sarah Nguyen| 19|Female|        10th|              7.4|
| 14|Melanie Phillips| 17|Female|         9th|              8.7|
| 17|    Matthew Webb| 15|Female|         8th|              5.4|
| 19|      Amy Greene| 19|Female|         9th|              7.9|
| 29|  Tammy Gallegos| 17|Female|        10th|              6.6|
| 60|  Kelly Carrillo| 17|Female|         7th|              6.3|
| 67|   Zachary Parks| 16|Female|        10th|              7.4|
| 76|      Brenda Lee| 13|Female|         8th|              6.5|
| 77| Jennifer Levine| 17|Female|         9th|              5.8|
| 82|      Andrew Kim| 19

### II. Average Sleep vs. Addiction

In [12]:
spark.sql(
    """
    select addiction_stage, round(avg(sleep_hours), 2) as avg_sleep
    from 
        (select id, name, age, gender, school_grade, daily_usage_hours, sleep_hours,
            (case 
                when addiction_level >= 0 and addiction_level < 3 then "light"
                when addiction_level >= 3 and addiction_level < 7 then "moderate"
                when addiction_level >= 7 then "severe"
            end) as addiction_stage     
        from teen_dataset) t
    group by addiction_stage
    """
).show()

+---------------+---------+
|addiction_stage|avg_sleep|
+---------------+---------+
|          light|      8.2|
|       moderate|     7.17|
|         severe|     6.39|
+---------------+---------+



### III. Screen Time Before Bed Impact
The idea is that I am going to compare `academic_performance` of teens who have `screen_time_before_bed` > 2 hours vs. those with less.

In [13]:
spark.sql(
    """
    select 
        watch_status, 
        round(avg(academic_performance), 2) as avg_performance
    from 
        (select 
            (case 
                when screen_time_before_bed > 2 then "watcher" 
                else "non_watcher" 
            end) as watch_status, 
            id, 
            name,
            academic_performance,
            screen_time_before_bed
        from teen_dataset
        order by screen_time_before_bed desc) t
    group by watch_status
    """
).show()

+------------+---------------+
|watch_status|avg_performance|
+------------+---------------+
|     watcher|          74.28|
| non_watcher|          74.71|
+------------+---------------+



### IV. Phone Purpose Distribution

In [14]:
df.select('phone_usage_purpose').distinct().show()

+-------------------+
|phone_usage_purpose|
+-------------------+
|          Education|
|             Gaming|
|           Browsing|
|              Other|
|       Social Media|
+-------------------+



In [15]:
spark.sql(
    """
    select 
        phone_usage_purpose,
        round(avg(academic_performance)) as avg_performance,
        round(avg(self_esteem)) as avg_esteem,
        count(*) as number_of_users
    from teen_dataset
    group by phone_usage_purpose 
    """
).show()

+-------------------+---------------+----------+---------------+
|phone_usage_purpose|avg_performance|avg_esteem|number_of_users|
+-------------------+---------------+----------+---------------+
|          Education|           75.0|       6.0|            383|
|             Gaming|           76.0|       6.0|            390|
|           Browsing|           75.0|       6.0|            440|
|              Other|           74.0|       6.0|            423|
|       Social Media|           75.0|       5.0|            387|
+-------------------+---------------+----------+---------------+



### V. Weekend vs. Weekday Usage (indicating binge usage)

In [16]:
spark.sql(
    """
    select 
        name,
        weekend_usage_hours,
        daily_usage_hours
    from teen_dataset
    where weekend_usage_hours > 2 * daily_usage_hours
    """
).show()

+--------------------+-------------------+-----------------+
|                name|weekend_usage_hours|daily_usage_hours|
+--------------------+-------------------+-----------------+
|     Shannon Francis|                8.7|              4.0|
|        Edward Avila|                9.1|              3.0|
|    Michael Williams|                6.7|              3.0|
|    Patricia Johnson|                5.7|              2.7|
|      Cameron Hansen|                6.2|              1.7|
|         Brandi King|                5.8|              1.0|
|    Kathleen Hoffman|                7.6|              3.4|
|         Maria Ochoa|                9.3|              3.8|
|       Brian Coleman|                5.3|              0.7|
|    Gregory Martinez|                6.7|              3.0|
|     Valerie Blevins|                8.2|              2.8|
|        John Griffin|                6.7|              3.0|
|Christine Cunningham|                9.5|              2.6|
|           Amy Smith|  

### VI. Top Performers with High Usage

In [17]:
spark.sql(
    """
    select 
        id, 
        name, 
        gender, 
        daily_usage_hours, 
        academic_performance
    from 
        (select *
        from teen_dataset
        where daily_usage_hours > 6) t
    where academic_performance > 95
    order by daily_usage_hours desc
    """
).show()

+----+-------------------+------+-----------------+--------------------+
|  id|               name|gender|daily_usage_hours|academic_performance|
+----+-------------------+------+-----------------+--------------------+
| 583|       Julia Rogers|Female|             10.5|                  97|
| 978|     Alyssa Jenkins|  Male|              9.4|                  97|
| 596|      Brian Alvarez|Female|              9.1|                  99|
|2980|      Travis Harris|  Male|              8.7|                  98|
| 754|    Matthew Gardner|Female|              8.5|                  98|
| 239|      Kelly Vasquez|  Male|              8.4|                  99|
| 930|      Nina Humphrey|  Male|              8.4|                 100|
| 616|Christopher Jackson|  Male|              8.3|                  97|
| 705|         Dawn Solis|  Male|              8.3|                  98|
|  19|         Amy Greene|Female|              7.9|                  96|
| 800|      Zachary Moore|  Male|              7.8|

### VII. Exercise vs. Social Media

In [18]:
spark.sql(
    """
    select 
        activity_status, 
        count(*) as no_of_staudents,
        round(avg(time_on_social_media), 2) as avg_social_media
    from
        (select 
            *,
            case 
                when exercise_hours >= 0 and exercise_hours < 1 then "not_active"
                when exercise_hours >= 1 and exercise_hours < 3 then "kind_of_active"
                when exercise_hours >= 3 then "active"
            end as activity_status
        from teen_dataset) t
    group by activity_status
    """
).show()

+---------------+---------------+----------------+
|activity_status|no_of_staudents|avg_social_media|
+---------------+---------------+----------------+
| kind_of_active|           1047|            2.48|
|     not_active|            959|            2.53|
|         active|             17|            2.47|
+---------------+---------------+----------------+



### VII. Parental Control Effectiveness

In [19]:
df.select('parental_control').distinct().show()

+----------------+
|parental_control|
+----------------+
|               1|
|               0|
+----------------+



In [20]:
spark.sql(
    """
    select 
        control_status,
        round(avg(addiction_level), 2) as avg_addiction_level
    from 
        (select 
            *,
            case 
                when parental_control = 1 then 'parental_control'
                when parental_control = 0 then 'no_parental_control'
            end as control_status
        from teen_dataset) t
    group by control_status
    """
).show()

+-------------------+-------------------+
|     control_status|avg_addiction_level|
+-------------------+-------------------+
|   parental_control|               8.92|
|no_parental_control|               8.89|
+-------------------+-------------------+



### IX. High Anxiety – High Usage

In [21]:
spark.sql(
    """
    select
        id,
        name,
        age,
        gender,
        phone_checks_per_day,
        anxiety_level,
        dense_rank() over(order by anxiety_level desc) as rnk
    from teen_dataset
    where phone_checks_per_day > 100 and anxiety_level >= 7
    """
).show()

+----+-----------------+---+------+--------------------+-------------+---+
|  id|             name|age|gender|phone_checks_per_day|anxiety_level|rnk|
+----+-----------------+---+------+--------------------+-------------+---+
| 449| Lindsey Sheppard| 19|  Male|                 133|           10|  1|
|1240|    Vincent Logan| 15|  Male|                 119|           10|  1|
| 538|    Megan Morales| 13|  Male|                 117|           10|  1|
| 421|       Tony Smith| 14|Female|                 114|           10|  1|
| 668|     Joshua Young| 13|  Male|                 126|           10|  1|
|  35|      Amy Carroll| 16|  Male|                 123|           10|  1|
| 689|   Dana Rodriguez| 19|Female|                 134|           10|  1|
|  49|   Tyrone Cabrera| 18|  Male|                 119|           10|  1|
| 699|   Brittany Klein| 18|  Male|                 121|           10|  1|
|  88|    Brian Coleman| 19|Female|                 146|           10|  1|
| 719|     William Rich| 

### X. Depression vs. Gaming

In [22]:
spark.sql(
    """
    select 
        gamer_status, 
        count(*) as no_of_students,
        avg(depression_level) as avg_depression_lvl
    from
        (select
            *,
            case
                when time_on_gaming > 3 then 'gaymer_detected'
                else 'not_gaymer'
            end as gamer_status 
        from teen_dataset) t
    group by gamer_status
    """
).show()

+---------------+--------------+------------------+
|   gamer_status|no_of_students|avg_depression_lvl|
+---------------+--------------+------------------+
|gaymer_detected|           139| 5.402877697841727|
|     not_gaymer|          1884| 5.487791932059448|
+---------------+--------------+------------------+



### XI. Self Esteem Ranking

In [23]:
spark.sql(
    """
    select 
        school_grade, 
        round(avg(self_esteem), 2) as avg_esteem
    from teen_dataset
    group by school_grade
    order by avg_esteem
    """
).show()

+------------+----------+
|school_grade|avg_esteem|
+------------+----------+
|        12th|      5.38|
|        10th|      5.49|
|        11th|      5.58|
|         9th|      5.61|
|         8th|      5.64|
|         7th|      5.81|
+------------+----------+



### XII. Daily Usage by Location

In [24]:
spark.sql(
    """
    select 
        location, 
        avg(daily_usage_hours) as daily_usage
    from teen_dataset
    group by location
    order by daily_usage desc
    """
).show()

+-------------------+-----------+
|           location|daily_usage|
+-------------------+-----------+
|       Deborahmouth|       11.2|
|    North Henryfort|       11.0|
|       Derrickville|       11.0|
|    North Jacobstad|       10.6|
|         Walkerview|       10.6|
|        Desireeland|       10.5|
|      West Johntown|       10.3|
|        Butlerhaven|       10.2|
|      West Lisaberg|       10.2|
|         Turnerfurt|       10.2|
|  North Reginahaven|        9.7|
|         Jeremyport|        9.7|
|        West Justin|        9.5|
|     West Jasonstad|        9.5|
|   North Nathanstad|        9.5|
|     East Alejandro|        9.4|
|   Christophermouth|        9.4|
|  New Joannachester|        9.4|
|South Kimberlymouth|        9.4|
|      Margaretville|        9.3|
+-------------------+-----------+
only showing top 20 rows


In [25]:
spark.sql(
    """
    select 
        id, 
        name, 
        age, 
        gender, 
        location, 
        daily_usage_hours
    from (select 
            *, 
            dense_rank() over(partition by location order by daily_usage_hours desc) as rnk
        from teen_dataset) t
    where rnk = 1
    """
).show()

+----+----------------+---+------+--------------+-----------------+
|  id|            name|age|gender|      location|daily_usage_hours|
+----+----------------+---+------+--------------+-----------------+
|2318|    Justin Jones| 19|Female|     Aaronfurt|              2.2|
|2478|Cynthia Gonzalez| 13|  Male|     Aaronland|              4.6|
|1032|   Tracey Watson| 19|Female|   Abbottville|              7.8|
|2554|  Sherry Gregory| 18|  Male|   Abigailview|              6.2|
| 170|   Robert Decker| 13|  Male|     Adammouth|              4.6|
| 729|Michelle Jackson| 18|Female|     Adamsside|              4.5|
|1635|Cynthia Crawford| 16|Female|     Adamsstad|              6.8|
|1073|Crystal Whitaker| 16|Female|    Albertberg|              2.7|
|1988|   Melissa Scott| 18|Female|     Alecville|              4.7|
|  65|  Jaclyn Salazar| 17|  Male|Alejandraburgh|              8.7|
|1681|   Madison Nunez| 19|Female|Alejandramouth|              5.4|
|2116|      Emma Evans| 18|Female| Alexanderview

### XIII. Education-Oriented Students

In [26]:
spark.sql(
    """
    select 
        id, 
        name, 
        gender, 
        time_on_education, 
        social_gaming_combined
    from 
        (select 
            *,
            round(time_on_social_media + time_on_gaming, 2) as social_gaming_combined
        from teen_dataset) t
    where 
        time_on_education > social_gaming_combined
        and
        time_on_education > 2
    """
).show()

+----+-----------------+------+-----------------+----------------------+
|  id|             name|gender|time_on_education|social_gaming_combined|
+----+-----------------+------+-----------------+----------------------+
| 844|    Joseph Taylor|  Male|              2.2|                   1.9|
|1174|    Casey Sanford|  Male|              2.5|                   2.1|
|1324|  Sara Cunningham|Female|              2.3|                   1.3|
|1371|   Jeffrey Parker|  Male|              2.8|                   2.1|
|1584| Katherine Harris|Female|              2.1|                   1.6|
|1889|Christopher Moses|Female|              2.7|                   2.2|
|2040|       Renee Dean|Female|              2.9|                   1.7|
|2067|  Nicholas Snyder|  Male|              2.8|                   2.5|
|2767|       Gary Jones|  Male|              2.2|                   2.1|
|2797|       Sara Moore|  Male|              2.2|                   1.4|
|2955|      Tracy Perry|  Male|              2.3|  

### XIV. School-Life Balance

In [27]:
spark.sql(
    """
    select 
        id, 
        name,
        gender,
        daily_usage_hours,
        sleep_hours,
        exercise_hours,
        academic_performance
    from 
        (select 
            *
        from teen_dataset
        where 
            daily_usage_hours <= 3
            and
            sleep_hours >= 8
            and
            exercise_hours >= 1) t
    where academic_performance >= 81
    """
).show()

+----+--------------------+------+-----------------+-----------+--------------+--------------------+
|  id|                name|gender|daily_usage_hours|sleep_hours|exercise_hours|academic_performance|
+----+--------------------+------+-----------------+-----------+--------------+--------------------+
| 342|Christopher Hamilton|Female|              2.9|        8.1|           1.2|                  82|
| 476|      Kathryn Bowman|Female|              1.1|        9.0|           1.1|                  98|
| 504|      Elizabeth King|  Male|              2.8|        8.9|           2.0|                  91|
|1091|        Robert Quinn|Female|              0.0|        8.1|           1.5|                  82|
|1412|    Brandon Santiago|Female|              0.0|        8.8|           1.9|                  95|
|1724|      Lindsey Thomas|  Male|              2.7|        8.1|           2.0|                  90|
|2034|       Regina Burton|Female|              0.9|        8.8|           1.4|            

In [28]:
df.show(1)

+---+---------------+---+------+----------+------------+-----------------+-----------+--------------------+-------------------+--------------+-------------+----------------+-----------+----------------+----------------------+--------------------+---------------+--------------------+--------------+-----------------+-------------------+--------------------+-------------------+---------------+
| id|           name|age|gender|  location|school_grade|daily_usage_hours|sleep_hours|academic_performance|social_interactions|exercise_hours|anxiety_level|depression_level|self_esteem|parental_control|screen_time_before_bed|phone_checks_per_day|apps_used_daily|time_on_social_media|time_on_gaming|time_on_education|phone_usage_purpose|family_communication|weekend_usage_hours|addiction_level|
+---+---------------+---+------+----------+------------+-----------------+-----------+--------------------+-------------------+--------------+-------------+----------------+-----------+----------------+----------

### XV. Socially Active but Anxious

In [40]:
spark.sql(
    """
    select 
        id, 
        name, 
        age,
        gender,
        school_grade,
        social_interactions,
        rnk
    from 
        ( select 
            id, 
            name, 
            age,
            gender,
            school_grade,
            social_interactions,
            dense_rank() over(partition by school_grade order by social_interactions desc) as rnk
        from  
            (select *
            from teen_dataset
            where anxiety_level > 9) t) p
    where rnk <= 2
    """
).show()

+----+------------------+---+------+------------+-------------------+---+
|  id|              name|age|gender|school_grade|social_interactions|rnk|
+----+------------------+---+------+------------+-------------------+---+
| 249|      Spencer Cook| 15|Female|        10th|                 10|  1|
|1500|        Debra Owen| 19|  Male|        10th|                 10|  1|
|1694|      Tracy Sparks| 16|Female|        10th|                 10|  1|
| 846|         David Cox| 18|Female|        10th|                  9|  2|
|1571|     Robert Hanson| 19|Female|        10th|                  9|  2|
|2040|        Renee Dean| 17|Female|        10th|                  9|  2|
|2154|   Kevin Fernandez| 17|Female|        10th|                  9|  2|
|2175|      Lance Snyder| 16|Female|        10th|                  9|  2|
|  12|     Ruben Walters| 14|  Male|        11th|                 10|  1|
| 305| Michelle Williams| 18|Female|        11th|                 10|  1|
| 980|     Angela Walker| 18|Female|  